In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import torch
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import neural_network_lyapunov
import neural_network_lyapunov.relu_system as relu_system
import neural_network_lyapunov.lyapunov as lyapunov
import neural_network_lyapunov.pybullet_data_generation as pybullet_data_generation
import neural_network_lyapunov.dynamics_learning as dynamics_learning
import neural_network_lyapunov.encoders as encoders

import neural_network_lyapunov.test.train_2d_lyapunov_utils as train_2d_lyapunov_utils

def urdf_path(file):
    return os.path.join(os.path.dirname(neural_network_lyapunov.__file__), "urdf", file)

## Pendulum Settings

In [ ]:
dtype = torch.float64

# data
world_cb = pybullet_data_generation.load_urdf_callback(urdf_path("pendulum.urdf"))
joint_space = True

pybullet_x_lo = torch.tensor([-np.pi, -5.], dtype=dtype)
pybullet_x_up = torch.tensor([np.pi, 5.], dtype=dtype)
pybullet_noise = torch.tensor([.1, .1])
dataset_dt = .1
dataset_N = 5
grayscale = True
image_width = 48
image_height = 48
camera_eye_position = [0, -3, 0]
camera_target_position = [0, 0, 0]
camera_up_vector = [0, 0, 1]

# training
num_samples = 100
batch_size = 60
validation_ratio = .01
validation_rollouts_N = 15
validation_max_rollouts = 100

dyn_learning_opt = dynamics_learning.DynamicsLearningOptions()

dyn_learning_opt.dynynamics_loss_weight = 10.
dyn_learning_opt.lyapunov_loss_at_samples_weight = 1.
dyn_learning_opt.lyapunov_loss_weight = 1.
dyn_learning_opt.equilibrium_loss_weight = 10.

dyn_learning_opt.V_lambda = 0.01
dyn_learning_opt.V_eps = 0.1

# encoder-decoder
encoder_class = encoders.LinearEncoder1
decoder_class = encoders.LinearDecoder1
use_bce = True
use_variational = False
z_dim = 5
z_lo = -1. * torch.ones(z_dim, dtype=dtype)
z_up = 1. * torch.ones(z_dim, dtype=dtype)

# dynamics nn
dyn_nn_width = 20
dyn_nn_depth = 1

# lyapunov nn
lyap_nn_width = 20
lyap_nn_depth = 1

## Cubes Settings

In [ ]:
def load_falling_cubes_callback():
    def cb(pb):
        plane_id = pb.loadURDF(urdf_path("plane_white.urdf"), flags=pb.URDF_USE_SELF_COLLISION)
        pos = [0, 0, .25]
        orn = pb.getQuaternionFromEuler([0, 0, 0])
        cube1_id = pb.loadURDF(urdf_path("cube_blue.urdf"), pos, orn)
        pos = [0.03, 0, 0.025]
        cube2_id = pb.loadURDF(urdf_path("cube_red.urdf"), pos, orn)
        pos = [-.075, 0, 0.025]
        cube3_id = pb.loadURDF(urdf_path("cube_red.urdf"), pos, orn)
        return cube1_id
    return cb

dtype = torch.float64

# data
world_cb = load_falling_cubes_callback()
joint_space = False

pybullet_x_lo = torch.tensor([-.1, 0, .15, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=dtype)
pybullet_x_up = torch.tensor([.1, 0, .15, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=dtype)
pybullet_noise = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=dtype)
dataset_dt = .05
dataset_N = 10
grayscale = False
image_width = 128
image_height = 128
camera_eye_position = [0, -.5, .15]
camera_target_position = [0, 0, .1]
camera_up_vector = [0, 0, 1]

# training
num_samples = 100
batch_size = 60

validation_ratio = .01
validation_rollouts_N = 10
validation_max_rollouts = 10

dyn_learning_opt = dynamics_learning.DynamicsLearningOptions()

dyn_learning_opt.dynynamics_loss_weight = 10.
dyn_learning_opt.lyapunov_loss_at_samples_weight = 1.
dyn_learning_opt.lyapunov_loss_weight = 1.
dyn_learning_opt.equilibrium_loss_weight = 10.

dyn_learning_opt.V_lambda = 0.01
dyn_learning_opt.V_eps = 0.1

# encoder-decoder
encoder_class = encoders.CNNEncoder1
decoder_class = encoders.CNNDecoder1
use_bce = True
use_variational = False
z_dim = 8
z_lo = -1. * torch.ones(z_dim, dtype=dtype)
z_up = 1. * torch.ones(z_dim, dtype=dtype)

# dynamics nn
dyn_nn_width = 20
dyn_nn_depth = 1

# lyapunov nn
lyap_nn_width = 20
lyap_nn_depth = 1

## Generating data

In [ ]:
pbsg = pybullet_data_generation.PybulletSampleGenerator(world_cb, joint_space, image_width=image_width, image_height=image_height,
                                                        grayscale=grayscale, dtype=dtype,
                                                        camera_eye_position=camera_eye_position,
                                                        camera_target_position=camera_target_position,
                                                        camera_up_vector=camera_up_vector)
x_data, x_next_data, X_data, X_next_data = pbsg.generate_dataset(pybullet_x_lo, pybullet_x_up, dataset_dt, dataset_N, num_samples)

In [ ]:
# plot some of the data
i = np.random.choice(X_data.shape[0], 1)[0]
pybullet_data_generation.show_sample(X_data[i,:], X_next_data[i,:])

In [ ]:
plt.plot(x_data)
plt.show()

In [ ]:
i = np.random.choice(X_data.shape[0], 1)[0]
X_traj, x_traj = pbsg.generate_rollout(x_data[i,:], dataset_dt, 5)
for n in range(X_traj.shape[0]):
    pybullet_data_generation.show_sample(X_traj[n, :])

In [ ]:
x_data_ = dynamics_learning.add_noise(x_data, pybullet_noise)
x_next_data_ = dynamics_learning.add_noise(x_next_data, pybullet_noise)
x_train_dataloader, x_validation_dataloader = dynamics_learning.get_dataloaders(x_data_, x_next_data_, batch_size, validation_ratio)

X_train_dataloader, X_validation_dataloader = dynamics_learning.get_dataloaders(X_data, X_next_data, batch_size, validation_ratio)

X_rollouts, x_rollouts = dynamics_learning.dataloader_to_rollouts(pbsg, x_validation_dataloader, dataset_dt, validation_rollouts_N,
                                                                  max_rollouts=validation_max_rollouts)

## Learning in state space

In [ ]:
dyn_nn_model = dynamics_learning.get_ff_network(dtype, z_dim, z_dim, dyn_nn_width, dyn_nn_depth)
lyap_nn_model = dynamics_learning.get_ff_network(dtype, z_dim, 1, lyap_nn_width, lyap_nn_depth)

relu_sys = relu_system.AutonomousReLUSystem(dtype, z_lo, z_up, dyn_nn_model)
lyap = lyapunov.LyapunovDiscreteTimeHybridSystem(relu_sys, lyap_nn_model)

dyn_learner = dynamics_learning.StateSpaceDynamicsLearning(x_train_dataloader, x_validation_dataloader, relu_sys, lyap, dyn_learning_opt)

In [ ]:
dyn_learner.train(10, validate=True)

In [ ]:
# plot a random rollout
x_traj = dyn_learner.rollout(x_data[np.random.choice(x_data.shape[0], 1)[0], :], 100)
plt.plot(x_traj)
plt.show()

In [ ]:
# validation over rollouts
dyn_learner.rollout_validation(x_rollouts)

## Learning in image space

In [ ]:
dyn_nn_model = dynamics_learning.get_ff_network(dtype, z_dim, z_dim, dyn_nn_width, dyn_nn_depth)
lyap_nn_model = dynamics_learning.get_ff_network(dtype, z_dim, 1, lyap_nn_width, lyap_nn_depth)

relu_sys = relu_system.AutonomousReLUSystem(dtype, z_lo, z_up, dyn_nn_model)
lyap = lyapunov.LyapunovDiscreteTimeHybridSystem(relu_sys, lyap_nn_model)
encoder = encoder_class(z_dim, image_width, image_height, grayscale)
decoder = decoder_class(z_dim, image_width, image_height, grayscale)

dyn_learner = dynamics_learning.LatentSpaceDynamicsLearning(X_train_dataloader, X_validation_dataloader, relu_sys, lyap, dyn_learning_opt,
                                                            encoder, decoder, use_bce=use_bce, use_variational=use_variational)

In [ ]:
dyn_learner.train_encoder(500, validate=True, device='cuda')

In [ ]:
torch.save(dyn_learner.encoder, "encoder")
torch.save(dyn_learner.decoder, "decoder")

In [ ]:
dyn_learner.encoder = torch.load("encoder")
dyn_learner.decoder = torch.load("decoder")

In [ ]:
torch.save(dyn_learner.relu_system.dynamics_relu, "dynamics")

In [ ]:
dyn_learner.relu_system.dynamics_relu = torch.load("dynamics")

In [ ]:
# plot some of the data
i = np.random.choice(X_data.shape[0], 1)[0]
pybullet_data_generation.show_sample(X_data[i,:])
X_decoded, _, _ = dyn_learner.encode_decode(X_data[i,:].unsqueeze(0))
X_decoded = X_decoded.squeeze()
pybullet_data_generation.show_sample(X_decoded)

In [ ]:
dyn_learner.train(1000, validate=True)

In [ ]:
dyn_learner.lyapunov_loss_weight = 0.

In [ ]:
dyn_learner.lyapunov_loss_weight

In [ ]:
# plot a random rollout
i = np.random.choice(X_data.shape[0], 1)[0]
x_traj = dyn_learner.rollout(X_data[i, :], validation_rollouts_N)
for n in range(x_traj.shape[0]):
    pybullet_data_generation.show_sample(x_traj[n, :])

In [ ]:
x_traj, _ = pbsg.generate_rollout(x_data[i, :], dataset_dt, validation_rollouts_N)
for n in range(x_traj.shape[0]):
    pybullet_data_generation.show_sample(x_traj[n, :])

In [ ]:
# validation over rollouts
dyn_learner.rollout_validation(X_rollouts)